# 池化

本文主要参考以下资料来了解更多关于池化的基本概念。

- [What are Max Pooling, Average Pooling, Global Max Pooling and Global Average Pooling?](https://www.machinecurve.com/index.php/2020/01/30/what-are-max-pooling-average-pooling-global-max-pooling-and-global-average-pooling/)

创建 ConvNets 通常与池化层密切相关。更具体地说，我们经常看到如最大池化、平均池化和全局池化等的卷积层之外的层。它们是什么？为什么它们是必要的，它们如何帮助训练机器学习模型？以及如何使用它们？

首先，从概念层面来看池化操作。通过此分析展示池化层如何帮助这些模型中生成的空间层次结构。然后，确定四种类型的池化——最大池化、平均池化、全局最大池化和全局平均池化。

## 什么是池化操作？

假设正在训练一个卷积神经网络。目标是从数据集中对图像进行分类。神经网络中第一个卷积层执行的操作可以表示如下：

![](img/CNN-1.jpg)

该层的输入是有高度，宽度，并带有三个通道的图像。使用 3x3x3 核，对输入图像执行卷积运算，生成N个 所谓的“特征图”，大小为 $H_{fm}*W_{fm}$. 一个特征图学习图像中存在的一个特定特征。通过激活，这些特征图有助于训练期间的结果预测，也有助于新数据的预测。N 可以在开始训练过程之前配置。

假如图像是 32 x 32 像素，第一个卷积操作（假设步长为 1 并且没有任何padding）将产生 30 x 30 像素的特征图；设置N= 64，那么将在第一层中生成 64 张这样的图

## 对输入进行下采样

现在让我们退后一步，想想如果要成功训练 ConvNet，我们想要实现的目标。假设我们有一个图像分类器，主要目标是它对图像进行正确分类。

作为人类这样做时我们会同时查看细节和高级模式。

现在再来看看特征图的概念。

在第一层，将学习基于图像非常“具体”的方面的特征图。在这里，特征图由图像中非常低级的元素组成，例如曲线和边缘，也就是细节。然而，我们无法看到只有一个卷积层的更高级别的 模式。我们需要许多堆叠在一起来学习这些模式。这也称为构建空间层次结构。当从下到上移动时，良好的空间层次结构对数据进行了大量总结，它们就像一个金字塔。这是一个好的和一个坏的：

![](img/hierarchies.png)
良好的空间层次结构（左）与较差的空间层次结构（右）。

在 ConvNet 的卷积运算中，一个小块在整个输入图像上滑动，与它当前滑动的图像部分进行元素乘法。这是一个相对昂贵的操作。不能以更简单的方式完成吗？真的需要仅通过卷积建立层次结构吗？答案是否定的，池化操作证明了这一点。

## 引入池化

这是池化的一种定义：

池化基本上是“缩小”从前几层获得的图像。它可以比作缩小图像以降低其像素密度。

缩小规模！但它也以更简单的方式完成：通过执行诸如max的硬编码张量运算，而不是通过学习转换，我们不需要学习权重等相对昂贵的运算。通过这种方式，能以很小的成本获得一个不错的且可能有用的空间层次结构。

接下来介绍四种常见类型的池化操作：

- 最大池化；
- 平均池化；
- 全局最大池化；
- 全局平均池化。

先来看看Max Pooling。

## 最大池化

假设这是来自 ConvNet 的 4 x 4 像素特征图之一：

![](img/Max-Pooling.png)

如果我们想对其进行下采样（downscaling），可以使用称为“最大池化”的池化操作（更具体地说，这是二维最大池化）。在这个池化操作中，一个H×W “块”滑过输入数据，其中 H 是高度和 W是宽度。步幅（即它在滑动操作期间的步幅）通常等于池大小，因此其效果等于高度和宽度的减少。

对于每个块或“池”，max运算只涉及计算一个最值，像这样：

![](img/Max-Pooling-1.png)

对每个池都这样做，我们得到了一个很好的下采样结果，大大有利于我们需要的空间层次结构：

![](img/Max-Pooling-2.png)

### 最大池化如何有益于平移不变性

除了作为卷积层的廉价替代品之外，最大池化在 ConvNet 中非常有用还有另一个原因：平移不变性。

当模型具有平移不变性时，这意味着对象在图片中的位置无关紧要；无论如何它都会被认出。例如，如果我将手机靠近头或靠近口袋——这两次都应该是分类的一部分。

可以想象，在模型中实现平移不变性极大地提高了其预测能力，因为不再需要提供对象精确位于某个所需位置的图像。相反，可以只提供包含对象的大量图像，并可能获得性能良好的模型。

最大池化如何在神经网络中实现平移不变性？

假设我们有一个单像素的对象——这有点奇怪，因为对象通常是多像素的，但它有利于我们的解释。对象具有最高的对比度，因此为输入图像中的像素生成高值。假设上图红色部分(0, 4)处的4是我们选择的像素。正如我们所见，使用最大池化时，它仍然包含在输出中。

现在想象这个对象，也就是 4，不在 (0, 4) 处，而是在 (1, 3) 处。它会从模型中消失吗？不是。相反，最大池化层的输出仍然是 4。因此，对象在红色块中的位置并不重要，因为无论如何它都会被“捕获”。

这就是为什么最大池化意味着平移不变性以及它真正有用的原因，除了相对耗费小。

注意到，然而，如果对象是在任何非红色区域中，只有当没有什么更大的像素值（这是所有元素的情况下！）时，对象才会被识别。因此，如果只提供对象始终位于非常小的 区域内的图片，则最大池化不会产生平移不变性。但是，如果数据集足够多，并且对象位于不同的位置，则最大池化确实有利于模型的性能。

### 为什么最大池化是最常用的池化操作

接下来，我们将看看平均池化，这是另一个池化操作。它可以用作 Max Pooling 的替代品。但是，当查看神经网络理论时，会发现 Max Pooling 一直是首选。

为什么会这样？

论点相对简单：由于感兴趣的对象可能产生最大的像素值，因此在某些块中取最大值比取平均值更有趣。

## 平均池化

另一种类型的池化层是平均池化层。这里，不是一个max值，而是avg计算每个块的，正如所看到的，输出也不同——与最大池化相比没有那么极端：

![](img/Average-Pooling-1.png)

平均池化与最大池化的不同之处在于它保留了关于块或池的“不太重要”元素的大量信息。Max Pooling 只是通过选择最大值来丢弃它们，而Average Pooling 会将它们混合在一起。这在此类信息有用的各种情况下都很有用。

### 为什么要考虑平均池化？

在互联网上，可以找到许多赞成和反对平均池化的论点，通常建议使用最大池化作为替代方案。首先，答案处理上述差异。

例如：

> 因此，为了回答您的问题，我认为平均池化比最大池化没有任何显着优势。但是，可能在某些情况下，最大池过滤器中的差异不显着，两个池都会给出相同类型的结果。但在极端情况下，最大池化肯定会提供更好的结果。

> 我要添加一个额外的论点——最大池化层在保持局部化方面更糟糕。

因此，唯一正确的答案是：这完全取决于要解决的问题。

如果对象的位置不重要，Max Pooling 似乎是更好的选择。如果重要，似乎使用平均池化可以获得更好的结果。

## 全局最大池化

另一种类型的池化层是全局最大池化层。在这里，我们将池大小设置为等于输入大小，以便max将整个输入的 计算为输出值：

![](img/Global-Max-Pooling-1.png)

全局池化层可用于多种情况。首先，它可用于降低某些卷积层输出的特征图的维数，以替换分类器中的 Flattening 层，有时甚至是Dense层。它还可以用于单词识别。这是由于它允许检测噪声的特性，“大输出”（例如上面示例中的值 9）。然而，这也是全局最大池化的缺点之一，

与常规池一样，我们接下来介绍全局平均池化。

## 全局平均池化

在应用全局平均池化时，池大小仍然设置为层输入的大小，但不是最大值，而是取池的平均值：

![](img/Global-Average-Pooling-3.png)

它们通常用于替换分类器中的全连接或Dense连接层。相反，该模型以一个卷积层结束，该层生成与目标类数量一样多的特征图，并对每个特征图应用全局平均池化，以便将每个特征图转换为一个值。由于特征图可以识别输入数据中的某些元素，最后一层中的图可以有效地学习“识别”该架构中特定类的存在。通过将全局平均池化生成的值输入到Softmax 激活函数中，将再次获得想要的多类概率分布。

更重要的是，这种方法可能会提高模型性能，因为“分类器”对“特征提取器”的原生性（它们都是卷积而不是卷积/密集），并减少过度拟合，因为没有参数在全局平均池化层中学习。

## PyTorch API 中的池化层

### 最大池化

最大池化具有一维、二维和三维变体。[一维变体](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool1d.html)可以与 Conv1D 层一起使用，其参数是 kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False.

In [1]:
import torch
from torch import nn

In [2]:
m = nn.MaxPool1d(3, stride=2)
input = torch.randn(20, 16, 50)
output= m(input)
output.shape

torch.Size([20, 16, 24])

在这里，池大小可以通过 kernel_size 设置，可以应用stride和padding，使用 strides，如果None 将默认为 kernel_size。

Max Pooling 也可[用于 2D 数据](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d)，可与 Conv2D 一起用于空间数据.

In [3]:
# pool of square window of size=3, stride=2
m = nn.MaxPool2d(3, stride=2)
input = torch.randn(20, 16, 50, 32)
output = m(input)
output.shape

C:\Users\hust2\miniconda3\envs\HDB\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([20, 16, 24, 15])

In [4]:
# pool of non-square window
m = nn.MaxPool2d((3, 2), stride=(2, 1))
output = m(input)
output.shape

torch.Size([20, 16, 24, 31])

### 平均池化

对于平均池化，API 与最大池化基本一致，所以不再一一记录了：

In [5]:
# pool with window of size=3, stride=2
m = nn.AvgPool1d(3, stride=2)
m(torch.tensor([[[1.,2,3,4,5,6,7]]]))

tensor([[[2., 4., 6.]]])

### 全局最大池化

由于池形状等于输入形状的全局池化层的独特结构，它们在 Tensorflow API 中的表示非常简单，但是 PyTorch 中并没有直接的对应项，不过可以使用adaptive pooling layer。

先看看 AdaptiveMaxPool

In [6]:
m = nn.AdaptiveMaxPool1d(5)
input = torch.randn(1, 64, 8)
output = m(input)
output.shape

torch.Size([1, 64, 5])

In [7]:
# target output size of 5x7
m = nn.AdaptiveMaxPool2d((5,7))
input = torch.randn(1, 64, 8, 9)
output = m(input)
output.shape

torch.Size([1, 64, 5, 7])

In [8]:
# target output size of 7x7 (square)
m = nn.AdaptiveMaxPool2d(7)
input = torch.randn(1, 64, 10, 9)
output = m(input)
output.shape

torch.Size([1, 64, 7, 7])

In [9]:
# target output size of 10x7
m = nn.AdaptiveMaxPool2d((None, 7))
input = torch.randn(1, 64, 10, 9)
output = m(input)
output.shape

torch.Size([1, 64, 10, 7])

可以看到，直接定义pooling输出的W和H的大小，所以对于全局最大池化，可以这样：

In [10]:
x = torch.randn(3, 4, 14, 14)
globalmaxpool = nn.AdaptiveMaxPool2d(1)
out = globalmaxpool(x)
out.shape

torch.Size([3, 4, 1, 1])

### 全局平均池化

对于全局平均池化，同样的情况：

In [11]:
x = torch.randn(3, 4, 14, 14)
globalmaxpool = nn.AdaptiveAvgPool2d(1)
out = globalmaxpool(x)
out.shape

torch.Size([3, 4, 1, 1])

### Create a model

In [12]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

可以看到添加了 Conv2D 层，然后是具有(2, 2)池大小的 MaxPooling2D 层——每次有效地将输入减半。该层有助于提高模型的泛化能力。

就是这样！将池化层应用于模型非常简单🙂

## 概括

了解了什么是池化层以及它们为何对机器学习项目有用。在一般性讨论之后，更详细地研究了最大池化、平均池化、全局最大池化和全局平均池化。

之后是实践部分——介绍框架中池化层的 API 表示，最后，提供了一个使用 MaxPooling2D 层将最大池化添加到 ConvNet 的示例。